# Task 1: Yelp Rating Prediction via Prompting (Gemini)

Prompt-based 1–5 star prediction using Gemini (no OpenAI). Includes multiple prompt strategies and evaluation.

**Sections**
- Setup (Gemini client)
- Data sampling
- Prompt strategies (>=3)
- Inference & JSON validation
- Evaluation & comparison
- Save results
- Notes / To-Do


In [ ]:
# Setup: imports and config
import os, json, random
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict
import google.generativeai as genai

# Env
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY') or os.getenv('GEMINI_API_KEY')
MODEL_NAME = os.getenv('MODEL_NAME', 'gemini-1.5-flash')

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print('⚠️ GOOGLE_API_KEY/GEMINI_API_KEY not set; inference will be skipped.')

data_path = Path('data')
results_path = Path('results')
for p in (data_path, results_path):
    p.mkdir(exist_ok=True)

print('Python ready. Data dir:', data_path.resolve())


## Prompt Strategy 1 (Baseline)
Simple instruction-only prompt that asks for star rating and brief reasoning.

In [ ]:
baseline_prompt = """
You are a strict JSON generator. Given a Yelp review, output a JSON object with:
- predicted_stars: integer 1-5
- explanation: brief reason

Return **only** JSON.
Review: {review_text}
"""
print(baseline_prompt)


## Prompt Strategy 2 (Chain-of-Thought Lite)
Adds lightweight reasoning before final JSON.

In [ ]:
cot_prompt = """
You are an assistant that reasons step-by-step then outputs JSON only.
Steps:
1) Briefly reason about sentiment, service, food, ambiance, price.
2) Decide 1-5 stars.
3) Output JSON only: {"predicted_stars": <int>, "explanation": "..."}

Review: {review_text}
"""
print(cot_prompt)


## Prompt Strategy 3 (Self-Check)
Adds validation to correct JSON/rating if needed.

In [ ]:
selfcheck_prompt = """
You are a rating model with a self-check step.
- Draft a rating 1-5 with a short reason.
- If draft rating not in 1-5 or JSON invalid, fix it.
- Return final JSON only: {"predicted_stars": <int>, "explanation": "..."}

Review: {review_text}
"""
print(selfcheck_prompt)


## Data Sampling
Loads `data/yelp_sample.csv` (200 rows sampled). Replace file if you want a different sample.

In [ ]:
dataset_path = data_path / 'yelp_sample.csv'
if not dataset_path.exists():
    raise FileNotFoundError('data/yelp_sample.csv missing. Please provide a sample with columns text, stars.')

df = pd.read_csv(dataset_path)
sample_size = min(200, len(df))
df_sample = df.sample(sample_size, random_state=42) if len(df) > sample_size else df.copy()
print('Loaded rows:', len(df), 'Using sample:', len(df_sample))
df_sample.head()


## Inference Helper (Gemini)
Calls Gemini chat API and parses JSON; skips if no API key.

In [ ]:
import json

system_prompt = 'You output only JSON with predicted_stars and explanation.'

def run_prompt(prompt_template: str, review_text: str):
    if not GEMINI_API_KEY:
        return {'predicted_stars': None, 'explanation': 'no_api_key'}
    prompt = prompt_template.format(review_text=review_text)
    model = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=system_prompt)
    resp = model.generate_content(prompt)
    content = resp.text
    try:
        return json.loads(content)
    except Exception:
        start = content.find('{')
        end = content.rfind('}')
        if start != -1 and end != -1 and end > start:
            try:
                return json.loads(content[start:end+1])
            except Exception:
                return {'predicted_stars': None, 'explanation': 'parse_error', 'raw': content}
        return {'predicted_stars': None, 'explanation': 'parse_error', 'raw': content}


## Evaluation Loop
Runs each strategy; skips inference if no API key.

In [ ]:
strategies = {
    'baseline': baseline_prompt,
    'cot': cot_prompt,
    'selfcheck': selfcheck_prompt,
}

results = []
for name, tmpl in strategies.items():
    for _, row in df_sample.iterrows():
        review_text = row['text']
        gt = int(row['stars'])
        pred_obj = run_prompt(tmpl, review_text)
        pred = pred_obj.get('predicted_stars')
        valid_json = pred is not None and isinstance(pred, (int, float)) and 1 <= pred <= 5
        results.append({
            'strategy': name,
            'review': review_text,
            'ground_truth': gt,
            'pred': pred,
            'valid_json': valid_json,
            'explanation': pred_obj.get('explanation', '')
        })

res_df = pd.DataFrame(results)
res_df.head()


## Metrics & Comparison

In [ ]:
def summarize(df):
    total = len(df)
    valid = df['valid_json'].mean()
    acc = (df['pred'] == df['ground_truth']).mean()
    return pd.Series({'samples': total, 'json_valid_rate': valid, 'accuracy': acc})

if len(res_df):
    summary = res_df.groupby('strategy').apply(summarize)
    print(summary)
else:
    summary = pd.DataFrame()
summary


## Save Results
Exports detailed predictions and summary to `results/`

In [ ]:
results_path.mkdir(exist_ok=True)
res_out = results_path / 'task1_predictions.csv'
sum_out = results_path / 'task1_summary.csv'
if len(res_df):
    res_df.to_csv(res_out, index=False)
    if 'summary' in locals() and not summary.empty:
        summary.to_csv(sum_out)
    print('Saved', res_out, 'and', sum_out)
else:
    print('No results to save (likely missing API key).')


## Notes / To-Do
- Set `GOOGLE_API_KEY` (or `GEMINI_API_KEY`) and optional `MODEL_NAME` (default: gemini-1.5-flash) before running inference.
- Replace `data/yelp_sample.csv` if you want a different sample.
- Consider adding retry/backoff and temperature sweeps.
- Include these outputs in the report.
